# How to read this document

This document is designed to showcase the different basic functionality of Hyperspy. For this practical you're intended to build your own notebook with the help of this document. 

We've given you a line spectrum (lisp) called "list_core_loss.hspy", its associated ADF profile: "ADF_core_loss.hspy", its associated low loss: "low_loss_core_loss.hspy" and its corresponding ADF survey image "". In this practical you are supposed to explore the data and understand what they represent. To guide you, here are a few questions :

- What is the scale of the observed object ? In what energy range, the EELS was measured ?
- What are the different core loss edges present in the data ? 
- How many phases can you find along the line spectra ?
- What elements are present in the different phases and in which proportions ?
- What happens when you fit the data using reference spectra ? Why ? How to improve on that ?

To succeed at this practical, answer those questions in a separate ipython notebook. If you want to go further, you are welcome to move to the next level.

**The jupyter notebook extensions are installed. Thus, you can navigate between sections using the left panel**

Not everything is intended to be read the from the first time. Optional comments along the document are marked by emojis: 
- ⭐ This is a tip, describing more in depth the inner workings of hyperspy.
- 🏠 It's an optional comment. Something that you may not use here but that you can use back to your lab.

The data were kindly given by: Cécile Marcelot.

# Introduction to Hyperspy

Hyperspy is a tool for the visualisation and manipulation of high-dimensional data. The main features of Hyperspy are the following:
- The Signal1D and Signal2D objects
- Domain-specific features for EELS, EDX, etc ...
- Model fitting
- Machine learning

This document will guide through the following steps : 
- Manipulation and visualisation of data using the Signal objects. 
- Various Data preprocessing such as energy alignement or spikes removal.
- How to perform a chemical map.
- How to perform a 1D and 3D fit.

A large part of this practical is inspired from the hyperspy documentation : http://hyperspy.org/hyperspy-doc/current/index.html

# 🏠 Installing Hyperspy with anaconda

This section is intended to help you to install hyperspy on your own computer in your own lab. **You can skip this part for the practical.**

Anaconda is python package manager dedicated to science. Using Anaconda it is possible to create virtual environments. You can then manage different virtual environments to isolate different distributions of python packages. This way you can keep stable python distributions safe from unstable ones.

We describe here the few steps required to create an environment with Hyperspy.

### 🏠 Install Anaconda

Depending on your operating system install anaconda : https://www.anaconda.com/products/individual

Then, launch the Anaconda Prompt, a console should appear.

### 🏠 Create a virtual environment

Run the command : ```conda create --name myenv``` (change "myenv" by any name you like.)

Your virtual environment is created but you're still in the default one called "base". You need to activate it by typing: ```conda activate myenv```

### 🏠 Installing packages

When installing a package there are two solutions: 
1. Either it is part of the already available libraries 
2. Either you can access it from anaconda forge

(1). Run the command : ```conda install package-name``` with for example: ```conda install scikit-learn```

(2). The available python packages are listed on : https://anaconda.org/ . After searching for the library you want, you can click on its name in bold. There will be a list of commands to type and execute to install the selected package. For Hyperspy it is: ```conda install -c conda-forge hyperspy```

### 🏠 installing ipython

The next few steps will enable you to run the virtual environment you created in ipython notebooks. In the Anaconda prompt run the following commands :
- ```conda install jupyter```
- ```conda install ipykernels```
- ```python -m ipykernel install --user --name myenv```

Now you can start the ipython notebook environment using: ```jupyter notebook```. You will be able to create ipython notebooks (top right corner) with the virtual environments you want. All the packages installed (and their versions) will be specific to that environment. 

# Import Hyperspy

⭐ We choose first a graphical backend that manage interactivity (```%matplotlib qt```).

In python the libraries have to be imported to use their functions. For clarity, hyperspy will be imported with the name "hs". 

In [1]:
%matplotlib qt
import hyperspy.api as hs

# Loading data

Hyperspy is able to load and manage data from many different open-source and proprietary formats, such as: 
- dm3, dm4
- emd, bcf
- tiff, jpg, etc...

For an exhaustive list : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#supported-formats

⭐ Without arguments ```hs.load()```  makes a file explorer window pop up. You can also load a list of files.

In [2]:
lisp = hs.load("lisp_core_loss.hspy")
ll = hs.load("low_loss_core_loss.hspy")
adf = hs.load("ADF_core_loss.hspy")
adf_survey = hs.load("ADF_survey_core_loss.hspy")

In [3]:
lisp.plot()

# Data structure and visualisation

The loaded data have a signal type, such as : `EELSSpectrum`, `EDSTEMSpectrum` or `EDSSEMSpectrum`.

Optionnally the data can have a title. 

The main feature of hyperspy signals is the dimension. The dimensions are either navigation or signal. For a spectrum line there is one navigation dimensions X and one signal dimension Z. It is noted : `(X|Z)`

⭐ If you want to developp a new application, hyperspy can support its implementation as a new signal type.

In [ ]:
print(lisp)

## I. Metadata

The Signal object has a metadata attribute. Some of it is used for signal specific function (e.g. beam energy). In this tutorial all the relevant metatadata is already filled in.

🏠 You can add new metadata categories this way : `lisp.metadata.Category1 = {}` and new metadata entries this way : `lisp.metadata.Category1.entry1 = 'entry1 value'`

⭐For some of the proprietary file formats, some of the metadata are automatically read and loaded by hyperspy. Be careful, what is loaded and what is not.

⭐If you want to build metadata by yourself, there are some helper function such as : `lisp.set_microscope_parameters()` or `lisp.add_elements()`

In [ ]:
print(lisp.metadata)

## I. Axes manager

The signal object has an axes_manager attribute to manage the calibrations of both the navigation axes and the signal axis. The Graphical User Interface (GUI) enables you to edit the values. 

⭐ Just typing `lisp.axes_manager` prints it. You can then acces the values using e.g. : `lisp.axes_manager[0].scale` or `lisp.axes_manager[axis_name].size`

In [ ]:
lisp.axes_manager.gui()

## I. Plotting

The plotting makes 2 windows pop-up. The first window contains an image where the vertical direction is the stack of different spectra (i.e. along the line spectrum) and the horizontal dimension is the spectral one with the high intensity features correspond to core loss edges. A single line (or spectrum) Region Of Interest (ROI) is displayed in the top left of the image. The second window represents the spectrum associated to the ROI. 

You can move this ROI using arrow keys.

In [ ]:
lisp.plot()

# Data Preprocessing

## Cropping

The crop function can be used to cut out parts of the data. **Be careful : it overwrites the data.** The syntax is as follows : `lisp.crop(axis,start,stop)`. Note that axis is either an integer or an axis name. 

For `start` and `stop`, the options are :
- Integer indices : They do not take into account the calibration and correspond to matrix indices.
- Float indices : They take into account the calibration.

⭐Using `.inav` for navigation axes or `.isig` for signal axes is a more powerful way to slice data. It supports integer indexing, float indices and even string indices with e.g. `lisp.inav["10 nm","20 nm"]`. It also supports advanced slicing such as `lisp.isig[::2]` or `lisp.inav[::,-1]`.



In [ ]:
lisp.crop(1,10.0,20.0)

## 🏠 Spikes removal

In this particular case there is no need to remove the cosmic X-rays. In general however, cosmic X-rays can be measured by the detector and their high intensity tends to produce outliers in the data. 
**This tool has a well documented GUI, using the integrated instructions it should be straightforward to remove X-ray spikes.**

In [ ]:
lisp.spikes_removal_tool()

# Chemical mapping

## Searching for core-loss edges

You can find the different edges in your data using the GUI tool provided by hyperspy. **Click on the help button to learn how to use it.** Once an energy range is selected, click on refresh table to get the information you want.

⭐For many functions of hyperspy calling `lisp.function()` starts a gui while `lisp.function(*args,**kwargs)` executes the function directly, skipping the gui.

In [ ]:
lisp.edges_at_energy()

## Background removal

There is an interactive background removal tool in hyperpsy you can call using : `lisp.remove_background()`. Below the cell where the function is executed, there is a "Help" button. It will explain you how to use the tool. **Be careful : Once you click on apply, the spim object is modified.**

For EELS, the Power Law background is the most adapted.

⭐ If you bypass the GUI, you can put the result in a new object. This can be done as such : `new_lisp = lisp.remove_background((energy_start,energy_end))`

In [ ]:
lisp.remove_background()

## Integrating peaks and generating profiles

**This step should be performed only on background subtracted spim.** 

## Span ROI

**Execute this cell before the next one.** 

🏠 You don't need to understand what happens in that cell for this practical. It aims at subclassing the SpanROI of hyperspy to add a new functionality. You're welcome to use both cells below in your lab.

In [ ]:
import numpy as np

class cst_SpanROI (hs.roi.SpanROI) : 
    def mapping (self,spim = None, out_map = None) :
        out_map.data = spim.isig[self.left:self.right].sum(axis = -1).data
        return out_map

## Profiles of spectral features

Two windows will pop out :
- One window reprensenting the sum of spectra of the line spectrum and a green interactive slider on top of it.
- Another window representing interactively the sum over the slice selected in the first window. 

In [ ]:
# Get safe initial coordinates for the ROI.
half_e = lisp.axes_manager[-1].offset + lisp.axes_manager[-1].scale*0.5*lisp.axes_manager[-1].size
third_e = lisp.axes_manager[-1].offset + lisp.axes_manager[-1].scale*0.333*lisp.axes_manager[-1].size

# Sum over all the spectra of the spim
full_spectrum = lisp.sum(axis = 0 )
# Initialize the ROI
spectrum_ROI = cst_SpanROI(third_e,half_e)

# Plot both the spectrum and the ROI (interactively)
full_spectrum.plot()
spectrum_ROI.interactive(full_spectrum)

# Initialize a Signal2D which will represent the map
out = hs.signals.Signal1D(np.zeros((lisp.axes_manager[0].size,)))
# Link the spim, the map and the SpanROI interactively
map=hs.interactive(spectrum_ROI.mapping,recompute_out_event = spectrum_ROI.events.changed,spim = lisp,out_map = out)
map.plot()

## Save your results

Please, do not hesitate to save your results.

In [ ]:
map.save("filename.hspy")

# 🏠 Converting `.hspy` data to Digital Micrograph (DM) readable format

You first need to install this plugin in DM : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#importrpl-digital-micrograph-plugin

Then you just need to save your data in `.rpl` format using hyperspy. It will produce `.rpl` and `.raw` files. Using the DM plugin you'll be able to open the `.rpl` file and read your data in DM.

In [ ]:
lisp.save("lisp.rpl")

# 1D Spectrum fitting

<a id='spectrum_fitting'></a>

Here the fitting is performed on the sum of spectra over the whole spectrum image. When the metadata are set correctly (it can be done using `lisp.set_microscope_parameters()` or `lisp.add_elements()`) the relevant components for an EELS fit are set automatically.

In this case we apply the fitting on the sum of spectra.

## Cross-sections 

For core-loss, there are two types of cross-section models : `"hydrogenic"` or `"Hartree-Slater"`. It is set using the keyword argument : `GOS =`.
- **Hydrogenic**: This a simple atomic model with hydrogenic wave functions where the screening of the electrons is taken into account with a constant. This model is suitable for K edges of light elements and can be adapted empirically for L edges.
- **Hartree-Slater**: This model is more accurate and uses an Hartree-Slater central field model. The elements are considered in their atomic form. Thus, the solid state effects are not taken into account. 
- For more details check: Hofer1991_cs_cl_eels.pdf

### Multiple scattering

If you have access to a low-loss spectrum, you can use to account for multiple scattering with the keyword argument : `ll = `. This feature may not work well in some cases. **Keep it disabled for a first try**.

Uncomment the corresponding code if you want to try this feature.

## Fitting options

**Comment and uncomment the different lines dis

- In most cases, it is important to enable the fine structure of core-loss edges for more accurate fitting. This is done using: `model.enable_fine_structure()`

- In some cases, the valence of elements can change compared to the tabulated one. It will change the energy of the edge onset (this shift is usually called a chemical shift). You can use the function `model.enable_free_onset_energy()` to also fit the energy onset value. This feature can greatly help you if your energy scale is not fully accurate.

- If you removed the background beforehand you should disable the background component with `model.disable_background()`

### Cells description

- In the first cell, we initialize the model.

- In the second cell, a window will pop up with in red the data and in blue the model (a flat line at the beginning). You can then tune the initial parameters of the gaussian using the interface below the second. It is all interactive.

In [ ]:
# For a 1D we need to sum over the navigation axis
spectrum = lisp.sum(axis = 0)
# ll_spectrum = ll.sum(axis = 0)

# We create the model
model = spectrum.create_model(GOS = "hydrogenic")#, ll = ll_spectrum)

model.enable_fine_structure()
model.enable_free_onset_energy()
# model.disable_background()

# Check what is in the model
print(model.components)

In [ ]:
model.enable_adjust_position()
model.gui()

## Fitting

To fit core-loss spectra it is better to use the function `model.smart_fit()` which is optimized for that task. If don't want to use the function `smart_fit` you can use: `model.fit()`

In [ ]:
model.smart_fit()

## Display the results

### Plot

This function shows the global result of the fitting procedure. Thus, it is a first check of the validity of the fit. 

In [ ]:
model.plot(plot_components=True,plot_edges = True)

### Quantify

The quantify function will print out a small summary of the intensity of the different edges. <a id='quantify'></a>

In [ ]:
model.quantify()

### Model parameters

A more detailed summary of the parameters obtained using fitting. The output can be quite long.

In [ ]:
model.print_current_values()

# Fitting using reference spectra

With these few cells, you can use 2 reference spectra to fit your spectrum images and thus make more accurate quantifications of your data. **The reference spectra have to share the same energy scale as the data.**

## Create a custom component. 

**You need to copy and execute that cell for this pratical. Even though you're welcome to ask, you don't need to understand what it does and how it works. 🏠** 

Hyperspy supports the creation of custom components. This is a powerful tool to design your own data analysis.

For this component, the first two arguments `ref_1` and `ref_2` are the reference spectra (as numpy arrays). They will not change during the fitting procedure. The last arguments `a_ref_1` and `a_ref_2` are the learned weights of `ref_1` and `ref_2`, respectively.   

In [ ]:
from hyperspy.component import Component

class ReferenceFitting(Component):
    def __init__(self, ref_1 = None, ref_2 = None, a_ref_1=1, a_ref_2=2):
        # Define the parameters
        # Note that the reference spectra are not parameters
        Component.__init__(self, ('a_ref_1', 'a_ref_2'))
        
        self.a_ref_1.value = a_ref_1
        self.a_ref_2.value = a_ref_2

        self.a_ref_1.bmin = 0.
        self.a_ref_1.bmax = None
        
        self.a_ref_2.bmin = 0.
        self.a_ref_2.bmax = None
        
        self.a_ref_1.grad = self.a_ref_1
        self.a_ref_2.grad = self.a_ref_2
        
        assert ref_1 is not None, "You need to define a first reference spectrum."
        assert ref_2 is not None, "You need to define a second reference spectrum."
        
        self.ref_1 = ref_1
        self.ref_2 = ref_2

    # Define the function as a function of the already defined parameters,
    # x being the independent variable value (e.g. energy scale)
    # In this particular case, there is no independant variable
    def function(self, x):
        a1 = self.a_ref_1.value
        a2 = self.a_ref_2.value
        return a1*self.ref_1 + a2*self.ref_2
    
    # Optionally, for faster execution, you can define the gradients of each parameter
    def grad_a_ref_1(self, x):
        """
        Returns d(function)/d(a_ref_1)
        """
        return self.ref_1

    def grad_a_ref_2(self, x):
        """
        Returns d(function)/d(a_ref_2)
        """
        return self.ref_2

## Loading the data and initializing model

The lisp was cut out to reduce the computation time. The `.data` attribute of the reference spectra are called to get their 1D numpy array. 

The fitting of the edges are disabled so that there is no conflict with the reference spectra. 

The `ReferenceFitting` component is added to the list of components of the model.

### References 

CrO : SP McBride, R Brydson, Journal of Materials Science, Volume 39, page 6723-6734 (2004)

Fe : A. Hähnel, J. Woltersdorf, Journal of Solid State Chemistry 182, 2961-2971 (2009)

EELS database : Philip Ewels, Thierry Sikora, Virginie Serin, Chris P. Ewels and Luc Lajaunie.
"A Complete Overhaul of the Electron Energy-Loss Spectroscopy and X-Ray Absorption Spectroscopy Database: eelsdb.eu."
Microscopy and Microanalysis

In [ ]:
ref1 = hs.load("Fe_ref.hspy")
ref2 = hs.load("CrO_ref.hspy")

comp = ReferenceFitting(ref1.data,ref2.data,1,1)
model = lisp.create_model()

model.disable_edges()
model.extend([comp])

## Fitting

The `model.multifit` function applies the fitting procedure, for each element of the navigation axis.  

In [ ]:
model.multifit()

## Plotting the results

- First cell : For 2D (and 3D) data, the function `model.plot` shows the sum of both the model and the spectra. It's also a good validity check. 

- Second cell : The `model.plot_results` is a more powerful function that shows the spatial distribution (here profiles) of the different fitting parameters. 

In [ ]:
model.plot()

In [ ]:
model.plot_results()